<a href="https://colab.research.google.com/github/rajghorui21/Environmental-Monitoring-Pollution-Control/blob/main/Environmental_Monitoring_and_Pollution_Control_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import pandas as pd

In [51]:
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt

In [52]:
file_path = "/content/air_quality_dataset.csv"
df = pd.read_csv(file_path)
df

,SITE_ID,Week,Year,DATEOFF,Ca,Cl,HNO3,HNO3 PPB,K,Mg,Na,NH4,NO3,SO2,SO2 PPB,SO4,TNO3,DATEON
0,ABT147,27,2023,7/11/2023 8:00:00 AM,0.042,0.032,0.759,0.295,0.129,0.022,0.055,0.279,0.088,0.258,0.098,0.915,0.834,7/04/2023
1,ABT147,28,2023,7/18/2023 8:00:00 AM,0.114,0.033,0.74,0.287,0.091,0.056,0.302,0.268,0.332,0.316,0.121,1.273,1.06,7/11/2023
2,ABT147,29,2023,7/25/2023 8:00:00 AM,0.062,0.033,0.648,0.251,0.074,0.02,0.07,0.276,0.128,0.294,0.112,0.705,0.766,7/18/2023
3,ABT147,30,2023,8/01/2023 8:00:00 AM,0.064,0.033,0.598,0.232,0.069,0.022,0.096,0.304,0.196,0.333,0.127,0.841,0.784,7/25/2023
4,ABT147,31,2023,8/08/2023 8:00:00 AM,0.059,0.176,0.348,0.135,0.065,0.033,0.208,0.366,0.382,0.331,0.126,0.596,0.724,8/01/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,YOS404,48,2023,12/05/2023 8:00:00 AM,0.041,0.015,0.406,0.158,0.018,0.008,0.037,0.064,0.091,-,-,0.231,0.491,11/28/2023
2099,YOS404,49,2023,12/12/2023 8:00:00 AM,0.018,0.015,0.182,0.071,0.011,0.004,0.02,0.058,0.055,-,-,0.15,0.234,12/05/2023
2100,YOS404,50,2023,12/19/2023 8:00:00 AM,0.014,0.015,0.128,0.05,0.009,0.002,0.006,0.039,0.027,-,-,0.1,0.152,12/12/2023
2101,YOS404,51,2023,12/26/2023 8:00:00 AM,0.038,0.013,0.14,0.054,0.021,0.005,0.013,0.054,0.056,-,-,0.147,0.194,12/19/2023


In [53]:
print("\n Dataset Info :")
print(df.info())


 Dataset Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103 entries, 0 to 2102
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SITE_ID   2103 non-null   object
 1   Week      2103 non-null   int64 
 2   Year      2103 non-null   int64 
 3   DATEOFF   2103 non-null   object
 4   Ca        2103 non-null   object
 5   Cl        2103 non-null   object
 6   HNO3      2103 non-null   object
 7   HNO3 PPB  2103 non-null   object
 8   K         2103 non-null   object
 9   Mg        2103 non-null   object
 10  Na        2103 non-null   object
 11  NH4       2103 non-null   object
 12  NO3       2103 non-null   object
 13  SO2       2103 non-null   object
 14  SO2 PPB   2103 non-null   object
 15  SO4       2103 non-null   object
 16  TNO3      2103 non-null   object
 17  DATEON    2103 non-null   object
dtypes: int64(2), object(16)
memory usage: 295.9+ KB
None


In [54]:
print("\n Dataset Description :")
print(df.describe())


 Dataset Description :
              Week    Year
count  2103.000000  2103.0
mean     39.451260  2023.0
std       7.510182     0.0
min      27.000000  2023.0
25%      33.000000  2023.0
50%      39.000000  2023.0
75%      46.000000  2023.0
max      52.000000  2023.0


In [55]:
print("\n Missing Values :")
print(df.isnull().sum())


 Missing Values :
SITE_ID     0
Week        0
Year        0
DATEOFF     0
Ca          0
Cl          0
HNO3        0
HNO3 PPB    0
K           0
Mg          0
Na          0
NH4         0
NO3         0
SO2         0
SO2 PPB     0
SO4         0
TNO3        0
DATEON      0
dtype: int64
